## Deneme

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import DatasetDict, Dataset
import re

In [ ]:
prefix = "data/tr_wiki67/trwiki-67."
file_paths = [ f"{prefix}{postfix}.raw" for postfix in ["test", "train", "val"] ]
file_paths

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download("punkt")

In [ ]:
example_sentence = "Türkiye-İran55 ilişkileri.55_55 - 987@"
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(example_sentence)
words

## Test for prepare.py script (get_raw_dataset functionality)

In [ ]:
import data.tr_wiki67.prepare as prepare

prepare.get_raw_dataset()

In [ ]:
from datasets import load_from_disk

ds = load_from_disk("data/tr_wiki67/raw_dataset")

In [ ]:
print(ds["titles"][21])
print(ds["examples"][21])
print(ds["length"][21])

In [ ]:
print(ds["titles"][21])
print(ds["examples"][21])
print(ds["length"][21])

## Build Train and Save Tokenizer

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFKC(),
     normalizers.Lowercase()]
)

# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.WhitespaceSplit(), 
                                                   pre_tokenizers.Digits(individual_digits=True),
                                                   pre_tokenizers.Punctuation()])


special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

trainer = trainers.WordPieceTrainer(vocab_size=25_000, special_tokens=special_tokens, min_frequency=5,
                                    continuing_subword_prefix="##", limit_alphabet=500)





In [ ]:
print(tokenizer.normalizer.normalize_str("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?"))


In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?")

In [ ]:
from data.tr_wiki67.prepare import get_raw_dataset

dataset = get_raw_dataset()

def get_example_batch():
    """
    generator func
    will be used when training tokenizer
    """
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["examples"]

In [ ]:
tokenizer.train_from_iterator(get_example_batch(), trainer=trainer)

In [ ]:
encoding = tokenizer.encode("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?")
print(encoding.tokens)

In [ ]:
print(encoding)
print(encoding.ids)
print(encoding.type_ids)
print(encoding.tokens)
print(encoding.offsets)
print(encoding.attention_mask)
print(encoding.special_tokens_mask)
print(encoding.overflowing)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]") 

print(cls_token_id, sep_token_id)


tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)]
)

tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
encoding = tokenizer.encode(" merhaba benim adım name..." , " AS gardaşım peki benim adım ne...")
print(encoding)
print(encoding.ids)
print(encoding.type_ids)
print(encoding.tokens)
print(encoding.offsets)
print(encoding.attention_mask)
print(encoding.special_tokens_mask)
print(encoding.overflowing)

In [ ]:
tokenizer.decode(encoding.ids)

In [ ]:
with open("data/tr_wiki67/trwiki-67.test.raw", "+r") as f:
    content = f.read()
    total_num_char = len(content)
    acc = len(tokenizer.encode(content).tokens)
    print(total_num_char / acc)

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer/tokenizer_55.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

In [ ]:
wrapped_tokenizer.is_fast

In [ ]:
wrapped_tokenizer.save_pretrained("fast_wordpiece")

In [ ]:
test = """4 Haziran 1980'de İzmir'de doğdu. Melih Özakat İlkokulu'na, Özel Çakabey Lisesi'nde ortaokula ve konservatuvarda liseye gitti. 7 yaşında okul hayatına başlamasıyla TRT ve İzmir Devlet Konservatuvarı'nın çocuk korolarında müzik eğitimine başlaması aynı zamana denk gelir. İlk geri vokal deneyimini 12 yaşında Sezen Aksu'yla beraber 1992 yılında gerçekleştirdi. Sezen Aksu'nun yanında solo ve vokal çalışmaları devam ederken okul hayatı İzmir'de devam etmekteydi.
İlk TV programına yine Sezen Aksu'yla 1992 yılında TRT’de çıktı. Geri vokal çalışmalarının yanı sıra bu programda solo bir şarkı da seslendirmesi kamuoyunda büyük ilgi gördü. Büyük yetenek olarak lanse edilen Tuğba Özerk'in sahne ve müzik hayatı tam anlamıyla böyle başladı denilebilir. Okul hayatı bittikten sonra İstanbul’a taşınan sanatçı aralarında Ege, Deniz Seki gibi isimler bulunan sanatçılara geri vokal yaptı. Daha sonra solo sahne çalışmalarına başlayan Özerk, 2002 yılının Aralık ayında "Dün Gibi" adlı ilk solo albümünü çıkardı. 'Aşk yarası' adlı şarkısıyla uzun süre radyolarda liste başlarında yer aldı. Albümle beraber oyunculuk yönünü gösterebilme fırsatı yakalayan sanatçı, atv'nin uzun süredir yayın yapmış olduğu dizisi "Böyle mi olacaktı"da bir sene kadar rol aldı. İlk albümünde bir tane bestesi olan Özerk üç senelik boşlukta birçok besteye de imza attı.6 Temmuz 2005 tarihinde 'Lo Lo Lo' adlı ikinci solo albümünü çıkardı.'Lo Lo Lo' adlı şarkısıyla uzun süre radyolarda liste başı oldu. İkinci albümünde bir tane bestesi olan iki senelik boşlukta birçok besteye de imza attı
"""
encoding = wrapped_tokenizer(test)
print(len(encoding.tokens()))   # our tokenizer does not have any limit-boundry about block_size
encoding.tokens()[:254]

## Prepare Data For (NSP-MLM) Task

In [1]:
from datasets import Dataset, load_from_disk, load_dataset
from nltk.tokenize import sent_tokenize
import os

In [ ]:
# nsp_mlm_dataset = { "a_b_text": [], "x": [], "y": [], "att_mask": [], "token_type": [], "isNext": [] }

isNext = None
x = []
y = []
attention_mask = []
token_type = []
token_id = []
a_b_text = []

block_size = 255
A_segment_flag = True

In [ ]:
raw_table = load_from_disk("data/tr_wiki67/raw_dataset")

In [407]:
def create_pairs(raw_row):
    # split example by sentences
    list_of_sentences = sent_tokenize(raw_row["examples"])
    A = []
    B = []

    for pair_index in range(0, len(list_of_sentences)-1):
        A.append(list_of_sentences[pair_index])
        B.append(list_of_sentences[pair_index + 1])

    return {"A": A, "B": B}


def create_a_b_table(raw_table_path):
    """
    TODO: ara kayıt işlemleri yap (save as arrow), hali hazırda var ise yükle ve döndür, examples, titles, length kolonlarını sil
    """
    # load raw dataset from disk
    raw_table = load_from_disk(raw_table_path)
    # # bu satır geçici! test'i hızlı yapabilmek için
    # raw_table = Dataset.from_dict(raw_table[:5_000])
    # every example needs at least 2 sentence
    raw_table_filtered = raw_table.filter(lambda row:  len(sent_tokenize(row["examples"], language="turkish")) > 1)


    # create table that contains A, B kols (problem: row of this table has list of sentences not just one sentence for each pair)
    a_b_list_table = raw_table_filtered.map(create_pairs)


    # 
    A, B = [], []
    for A_list, B_list in zip(a_b_list_table["A"], a_b_list_table["B"]):
        A += A_list
        B += B_list
    
    a_b_table = {"A":A, "B":B}
    a_b_table = Dataset.from_dict(a_b_table)

    return a_b_table


In [ ]:
raw_table_path = "data/tr_wiki67/raw_dataset"
ab_table_path = "deneme_ab_tablosu"


# if table is not already exists
if os.path.exists(ab_table_path) == False:
    print("table is not already exists, will be created and saved to disk...")
    
    # create table
    a_b_table = create_a_b_table(raw_table_path)

    # save created table
    a_b_table.save_to_disk(ab_table_path)

# ab table is already exists
else:
    print("table is already exists, will be loaded from disk...")
    # load table from disk
    a_b_table = load_from_disk(ab_table_path)



In [ ]:
a_b_table

In [ ]:
temp = a_b_table[:1000]

In [ ]:
for i in range(5):
    print(f"A: {temp['A'][i]}, \nB: {temp['B'][i]}\n\n")

## NSP-MLM PREP

In [1]:
from datasets import load_from_disk, Dataset
import os
import random

from torch.utils.data import DataLoader
import torch

import re

from tokenizers import (
    pre_tokenizers,
    processors,
    Tokenizer,
)

In [2]:
# GLOBAL VARS  
trwiki_path_prefix = "data/tr_wiki67/"
raw_table_path = trwiki_path_prefix + "raw_dataset"
ab_table_path = trwiki_path_prefix + "deneme_ab_tablosu"
nsp_mlm_table_path = trwiki_path_prefix + "nsp_mlm_table"
block_size = 255

tokenizer = Tokenizer.from_file("tokenizer_wordpiece/tokenizer.json")

cls_id = tokenizer.token_to_id("[CLS]")
sep_id = tokenizer.token_to_id("[SEP]")
pad_id = tokenizer.token_to_id("[PAD]")
mask_id = tokenizer.token_to_id("[MASK]")

# if table is not already exists
if os.path.exists(ab_table_path) == False:
    print("table is not already exists, will be created and saved to disk...")
    
    # create table
    a_b_table = create_a_b_table(raw_table_path)

    # save created table
    a_b_table.save_to_disk(ab_table_path)

# ab table is already exists
else:
    print("table is already exists, will be loaded from disk...")
    # load table from disk
    a_b_table = load_from_disk(ab_table_path)

# this chunk will be used for finding random word or sentence
sentences_list = a_b_table[:10_000]
sentences_list = sentences_list["A"] + sentences_list["B"]  # list + list -> list

table is already exists, will be loaded from disk...


In [3]:
def get_random_sentence():
    return sentences_list[random.randint(0, len(sentences_list) - 1)]

def get_random_word_tokens(num_tokens):
    limit = 100
    trial = 0
    pattern = r'\d'

    while trial < limit:
        trial += 1
        words = get_random_sentence().split(" ")
        for word in words:
            # if word is number like 1881, 2005, ...
            # jumpt to next word
            if re.search(pattern, word):
                # print("number!! ", word)
                continue

            cand_word_token_ids = tokenizer.encode(word, add_special_tokens=False).ids
            if len(cand_word_token_ids) == num_tokens:
                # print(word)
                return cand_word_token_ids
            
    # trial limit exceeded return None
    return None

In [4]:
t = get_random_word_tokens(1)
if  t is None:
    print(None)
else:
    print(t), print(tokenizer.decode(t))

[11970]
tanrıça


In [5]:
get_random_sentence()

'İsviçre, Avrupa Birliği üyesi olmamasına rağmen, "ne bir siyasi birimin bir devlet olabilmesi için tanınması gerekmiyor, ne de bir devletin diğerini tanıma yükümlülüğü bulunmadığını belirterek belirtilen bu ilkeye bağlı kalıyor.'

In [6]:
a_b_table

Dataset({
    features: ['A', 'B'],
    num_rows: 4119470
})

In [19]:
gecici_dataset = {"a":[5,3,66,6510,4,None,45,444,None]}
gecici_dataset

{'a': [5, 3, 66, 6510, 4, None, 45, 444, None]}

In [21]:
gecici_dataset = Dataset.from_dict(gecici_dataset)

AttributeError: 'Dataset' object has no attribute 'items'

In [24]:
gecici_dataset

Dataset({
    features: ['a'],
    num_rows: 9
})

In [25]:
gecici_dataset.filter(lambda x: x["a"] != None)

Filter:   0%|          | 0/9 [00:00<?, ? examples/s]

Dataset({
    features: ['a'],
    num_rows: 7
})

In [26]:
gecici_dataset

Dataset({
    features: ['a'],
    num_rows: 9
})

In [30]:
def del_none_filter(row):
    return row["X"] != None


def create_nsp_mlm_table(ab_row):
    
    isNext = random.random() > 0.5
    x = []  # input sequence
    y = []  # target sequence
    
    if isNext == False:
        rand_b = get_random_sentence()
        ab_row["B"] = rand_b

    encoding = tokenizer.encode(ab_row["A"], ab_row["B"])
    ab_len = len(encoding.tokens) 


    if ab_len > block_size:
        return {"X": None, "Y": None, "Attention_mask": None, "Token_type_id":None, "isNext": None}
    
    seq = tokenizer.decode(encoding.ids, skip_special_tokens=False).split(" ")

    for word in seq:

        if word == "[CLS]":
            x.append(cls_id)
            y.append(int(isNext))

        elif word == "[SEP]":
            x.append(sep_id)
            y.append(pad_id)
        
        else:
            tokens = tokenizer.encode(word, add_special_tokens=False).ids
            
            prob = random.random()
            if prob < 0.15 :


                prob_inner = random.random()
                if prob_inner < 0.8:
                    # mask condition
                    x += [mask_id for token in tokens]
                    y += tokens
                    # print("mask condition: ")
                    # print("x: ", tokenizer.decode([mask_id for token in tokens], skip_special_tokens=False))
                    # print("y: ", tokenizer.decode(tokens, skip_special_tokens=False))
                               
                elif prob_inner < 0.9:
                    # corrupt condition
                    random_word_tokens = get_random_word_tokens(num_tokens=len(tokens))

                    if random_word_tokens is None:
                        # could not found word that has the same token length as corrupted word
                        print("could not found word")
                        print("len of tokes: ", len(tokens))
                        return {"X": None, "Y": None, "Attention_mask": None, "Token_type_id":None, "isNext": None}
                    
                    x += random_word_tokens
                    y += tokens
                    # print("corrupt condition:")
                    # print("x: ", tokenizer.decode(random_word_tokens, skip_special_tokens=False))
                    # print("y: ", tokenizer.decode(tokens, skip_special_tokens=False))
                
                else:
                    # identity condition
                    x += tokens
                    y += tokens
                    # print("identity condition:")
                    # print("x: ", tokenizer.decode(tokens, skip_special_tokens=False))
                    # print("y: ", tokenizer.decode(tokens, skip_special_tokens=False))
                    
                
            else:
                x += tokens 
                y += [pad_id for token in tokens]
            

    assert len(x) == len(y), "[ERROR] there is some mistake, x and y should be the same length!"

    num_pad = block_size - len(x)
    x += [pad_id for each in range(num_pad)]
    y += [pad_id for each in range(num_pad)]
    
    first_sep_idx = x.index(sep_id)
    token_type_id = [ 0 if i <= first_sep_idx else 1 for i in range(len(x)) ]
    attention_mask = [ 0 if token_id == pad_id else 1 for token_id in x ]

    # print("\nX: ", tokenizer.decode(x, skip_special_tokens=False),
    #        "\nY: ", tokenizer.decode(y, skip_special_tokens=False),
    #        "\nAttention_mask: ", attention_mask, 
    #        "\nToken_type_id: ", token_type_id, 
    #        "\nisNext: ", isNext , "\n\n")
    
    # convert Tensor
    x_tensor = torch.tensor(x, dtype=torch.int16)
    y_tensor = torch.tensor(y, dtype=torch.int16)
    att_mask_tensor = torch.tensor(attention_mask, dtype=torch.int16)
    token_type_tensor = torch.tensor(token_type_id, dtype=torch.int16)
    isNext_tensor = torch.tensor(isNext, dtype=torch.int16)

    return {"X": x_tensor, "Y": y_tensor, "Attention_mask": att_mask_tensor, "Token_type_id":token_type_tensor, "isNext": isNext_tensor}


In [33]:
nsp_mlm_table = a_b_table.map(create_nsp_mlm_table).filter(del_none_filter).remove_columns(["A", "B"])

Map:   0%|          | 0/4119470 [00:00<?, ? examples/s]

could not found word
len of tokes:  9
could not found word
len of tokes:  6
could not found word
len of tokes:  9
could not found word
len of tokes:  8
could not found word
len of tokes:  16
could not found word
len of tokes:  10
could not found word
len of tokes:  10
could not found word
len of tokes:  8
could not found word
len of tokes:  11
could not found word
len of tokes:  10
could not found word
len of tokes:  9
could not found word
len of tokes:  7
could not found word
len of tokes:  9
could not found word
len of tokes:  15
could not found word
len of tokes:  8
could not found word
len of tokes:  10
could not found word
len of tokes:  7
could not found word
len of tokes:  10
could not found word
len of tokes:  7
could not found word
len of tokes:  8
could not found word
len of tokes:  9
could not found word
len of tokes:  12
could not found word
len of tokes:  10
could not found word
len of tokes:  9
could not found word
len of tokes:  8
could not found word
len of tokes:  7
co

Filter:   0%|          | 0/4119470 [00:00<?, ? examples/s]

In [34]:
nsp_mlm_table.save_to_disk(trwiki_path_prefix + "nsp_mlm_table")

Saving the dataset (0/17 shards):   0%|          | 0/4119470 [00:00<?, ? examples/s]

In [10]:
x = nsp_mlm_table["X"]

In [14]:
d = {"ss":x}

In [15]:
import pandas as pd
asd = pd.DataFrame(d)

In [17]:
asd.isnull().sum() !!!!

ss    1943
dtype: int64

In [31]:
nsp_mlm_table = nsp_mlm_table.filter(del_none_filter)

Filter:   0%|          | 0/4119470 [00:00<?, ? examples/s]

helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo
helloooooo


KeyboardInterrupt: 

In [9]:
nsp_mlm_table = load_from_disk(trwiki_path_prefix + "nsp_mlm_table") 

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

In [9]:
nsp_mlm_table[0]

{'A': "Thomas Lüthi (d. 6 Eylül 1986; Oberdiessbach), İnterwetten Paddock Moto2 takımı için, Moto2 Grand Prix Dünya Şampiyonası'nda yarışan İsviçreli profesyonel bir motosiklet yol yarışçısı.",
 'B': "2010 yılından bu yanan Moto2 Dünya Şampiyonası'nda yarışmaktadır.",
 'X': [2,
  5759,
  12618,
  2327,
  12,
  44,
  18,
  26,
  1883,
  21,
  4,
  4,
  26,
  31,
  16769,
  8797,
  10743,
  10543,
  13,
  16,
  1360,
  1253,
  3681,
  4678,
  510,
  14310,
  2943,
  1584,
  2575,
  522,
  22,
  2354,
  1072,
  16,
  2575,
  522,
  22,
  5297,
  7390,
  1490,
  3425,
  11,
  1573,
  12709,
  13776,
  2591,
  962,
  14869,
  1505,
  1694,
  9041,
  18,
  3,
  22,
  20,
  4,
  20,
  2911,
  4,
  22236,
  2575,
  522,
  22,
  1490,
  3425,
  11,
  1573,
  4643,
  11871,
  8859,
  506,
  18,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [93]:
deneme = nsp_mlm_table[:100]
deneme.keys()

dict_keys(['A', 'B', 'X', 'Y', 'Attention_mask', 'Token_type_id', 'isNext'])

In [97]:
x_tensor = torch.tensor(deneme["X"], dtype=torch.int16)
y_tensor = torch.tensor(deneme["Y"], dtype=torch.int16)
att_mask = torch.tensor(deneme["Attention_mask"], dtype=torch.int16)
token_type = torch.tensor(deneme["Token_type_id"], dtype=torch.int16)
isNext = torch.tensor(deneme["isNext"], dtype=torch.int16)

In [46]:
a.dtype

torch.float16

In [47]:
a.shape

torch.Size([100, 255])

In [ ]:
# önce bir dataloader kullanarak hız tespitinde bulun (+ shuffle beklediğin gibi mi olmuş test et: tekrar decode'la batch'i) [fonk'laştır veri görselleştirme mantığı]
# tensor dönüşümü yapılacak, o şekilde kaydedilecek (fonklar vs de ona göre kaydedilecek)
# split yapılabilecek

In [96]:
def print_examples(x, y, att_mask=None, token_type=None, is_next=None):
    if isinstance(x, torch.Tensor):
        x, y = x.tolist(), y.tolist()

    assert len(x) == len(y), "x and y are not equal..."

    for i in range(len(x)):
        print("x: ", tokenizer.decode(x[i], skip_special_tokens=False))
        print("y: ", tokenizer.decode(y[i], skip_special_tokens=False))

        if att_mask is not None:
            print("att_mask: ", att_mask[i])
        if token_type is not None:
            print("token_type: ", token_type[i])
        if is_next is not None:
            print("isNext: ", is_next[i])
        print("\n\n")


In [100]:
# print_examples(x_tensor, y_tensor, att_mask, token_type, isNext)
print_examples(x_tensor, y_tensor, is_next=isNext)

x:  [CLS] thomas lüthi ( d. 6 eylül 1 [MASK] [MASK] 6 ; oberdiessbach ), i̇nterwetten paddock moto 2 takımı için, moto 2 grand prix dünya şampiyonası ' nda yarışan i̇sviçreli profesyonel bir motosiklet yol yarışçısı. [SEP] 2 0 [MASK] 0 yılından [MASK] yanan moto 2 dünya şampiyonası ' nda yarışmaktadır. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 